In [1]:

# from dask.distributed import Client

# client = Client('tcp://localhost:8786')


In [2]:
# client

# File Setup

In [73]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import cartopy
import datetime
import pyproj
import rioxarray
from scipy.interpolate import griddata
import scipy as sp
import folium
import os
import folium
from ipywidgets import interact, Dropdown
from IPython.display import display, clear_output
from itertools import combinations
import matplotlib
# from matplotlib import animation
# import dask.array as da
from scipy.sparse import csr_matrix
#Define your datamesh token in evironment variables as DATAMESH_TOKEN or insert into argument below.
from oceanum.datamesh import Connector
import netCDF4 as nc
datamesh=Connector(token='3052e2bdd10904ae353ac54ed205df32bfcc20e2')

In [74]:
# import dask

In [75]:
# def create_dict_from_netcdf(netcdf_data):
#     data_dict = {}
#     for key, group in netcdf_data.items():
#         masked_array = np.ma.masked_array(group['data'])
#         masked_array.mask = group['mask']
#         data_dict[key] = masked_array
#     return(data_dict)


In [76]:
# Define the current and target CRS using EPSG codes
current_crs = pyproj.CRS("EPSG:2193")  
target_crs = pyproj.CRS("EPSG:4326")

transformer = pyproj.Transformer.from_crs(current_crs, target_crs, always_xy=True,allow_ballpark=False)

In [77]:
dsq=datamesh.query({"datasource":"oceanum_wave_glob05_era5_v1_grid",
                    "variables":["hs","dp"],
                    "timefilter":{"times":["2010-01-01","2011-01-01"]},
                    "geofilter":{
                        "type":"feature",
                        "geom":{
                            "type":"Feature","geometry":{"type":"Point","coordinates":[170.2,-35.3]}
                        }
                    }
               })

# Testing Atoms

Credit: https://github.com/lukepolson/youtube_channel/blob/main/Python%20Metaphysics%20Series/vid28.ipynb

In [8]:
# def get_delta_pairs(x, ids_pairs):
#     return np.diff(np.array([x[ids_pairs[:,0]], x[ids_pairs[:,1]]]).T, axis=1).ravel()

# def get_deltad_pairs(r, ids_pairs):
#     return np.sqrt(get_delta_pairs(r[0], ids_pairs)**2 + get_delta_pairs(r[1], ids_pairs)**2)

# def compute_new_v(v1, v2, r1, r2):
#     v1new = v1 - ((v1-v2)*(r1-r2)).sum(axis=0)/np.sum((r1-r2)**2, axis=0) * (r1-r2)
#     v2new = v2 - ((v1-v2)*(r1-r2)).sum(axis=0)/np.sum((r2-r1)**2, axis=0) * (r2-r1)
#     return v1new, v2new

# def motion(r, v, id_pairs, ts, dt, d_cutoff):
#     rs = np.zeros((ts, r.shape[0], r.shape[1]))
#     vs = np.zeros((ts, v.shape[0], v.shape[1]))
#     # Initial State
#     rs[0] = r.copy()
#     vs[0] = v.copy()
#     for i in range(1,ts):
#         ic = id_pairs[get_deltad_pairs(r, ids_pairs) < d_cutoff]
#         v[:,ic[:,0]], v[:,ic[:,1]] = compute_new_v(v[:,ic[:,0]], v[:,ic[:,1]], r[:,ic[:,0]], r[:,ic[:,1]])
        
#         v[0,r[0]>1] = -np.abs(v[0,r[0]>1])
#         v[0,r[0]<0] = np.abs(v[0,r[0]<0])
#         v[1,r[1]>1] = -np.abs(v[1,r[1]>1])
#         v[1,r[1]<0] = np.abs(v[1,r[1]<0])
        
#         r = r + v*dt
#         rs[i] = r.copy()
#         vs[i] = v.copy()
#     return rs, vs

In [9]:
# radius = 0.03
# n_particles = 16
# r = np.random.random((2,n_particles))
# ixr = r[0]>0.5 #right
# ixl = r[0]<=0.5 #left

# ids = np.arange(n_particles)
# ids_pairs = np.asarray(list(combinations(ids,2)))

# v = np.zeros((2,n_particles))
# v[0][ixr] = -500
# v[0][ixl] = 500

# rs, vs = motion(r, v, ids_pairs, ts=1000, dt=0.000008, d_cutoff=2*radius)

In [10]:
# v = np.linspace(0, 2000, 1000)
# a = 2/500**2
# fv = a*v*np.exp(-a*v**2 / 2)

# bins = np.linspace(0,1500,50)

In [11]:
# fig, axes = plt.subplots(1, 1, figsize=(10,10))

# def animate(i):
#     [ax.clear() for ax in axes]
#     ax = axes[0]
#     xred, yred = rs[i][0][ixr], rs[i][1][ixr]
#     xblue, yblue = rs[i][0][ixl],rs[i][1][ixl]
#     circles_red = [plt.Circle((xi, yi), radius=4*radius, linewidth=0) for xi,yi in zip(xred,yred)]
#     # circles_blue = [plt.Circle((xi, yi), radius=4*radius, linewidth=0) for xi,yi in zip(xblue,yblue)]
#     cred = matplotlib.collections.PatchCollection(circles_red, facecolors='red')
#     # cblue = matplotlib.collections.PatchCollection(circles_blue, facecolors='blue')
#     ax.add_collection(cred)
#     # ax.add_collection(cblue)
#     ax.set_xlim(0,1)
#     ax.set_ylim(0,1)
#     ax.tick_params(axis='x', labelsize=15)
#     ax.tick_params(axis='y', labelsize=15)
#     # ax = axes[1]
#     # ax.hist(np.sqrt(np.sum(vs[i]**2, axis=0)), bins=bins, density=True)
#     # ax.plot(v,fv)
#     # ax.set_xlabel('Velocity [m/s]')
#     # ax.set_ylabel('# Particles')
#     # ax.set_xlim(0,1500)
#     # ax.set_ylim(0,0.006)
#     # ax.tick_params(axis='x', labelsize=15)
#     # ax.tick_params(axis='y', labelsize=15)
#     fig.tight_layout()
    
    
# # ani = animation.FuncAnimation(fig, animate, frames=500, interval=50)
# # ani.save('ani.gif',writer='pillow',fps=30,dpi=100)

# Load DEM

In [12]:
# #Load datasource linz-wellington_2013-2014-dem_1m-2193
# linz_wellington_2013_2014_dem_1m_2193=datamesh.load_datasource('linz-wellington_2013-2014-dem_1m-2193')

In [13]:
# linz_wellington_2013_2014_dem_1m_2193 = datamesh.query({'datasource':'linz-wellington_2013-2014-dem_1m-2193',"geofilter":{"type":"bbox","geom":[174.536322,-41.442487,175.124777,-41.049083]}})

In [14]:
# linz_wellington_2013_2014_dem_1m_2193

In [15]:
df_vlms = pd.read_csv('our-changing-coast-insar_rates_searise.csv')
xr_dem = rioxarray.open_rasterio('dem_8m_2012_wgs//92L94-92L34.tif') #linz_wellington_2013_2014_dem_1m_2193#

In [16]:
# xr_dem = xr_dem.band_data

In [17]:
# np.array(dem_values)

In [18]:
dem_values = xr_dem.values
fill_value = np.min(dem_values)

dem_values[dem_values==fill_value] = np.nan
xr_dem.values = dem_values

In [19]:
# Create a regular grid of coordinates
res = 500# low res to make the initial calcs fast
x_regular = np.linspace(xr_dem['x'].min(), xr_dem['x'].max(), res)
y_regular = np.linspace(xr_dem['y'].min(), xr_dem['y'].max(), res)

# Interpolate data onto regular grid
data_interpolated = xr_dem[0,:,:].interp(y=y_regular, x=x_regular)

# Now you can use the interpolated data for transformation
new_DEM = xr.DataArray(data_interpolated,coords={"y":y_regular,"x":x_regular},dims=["y","x"])

In [20]:
new_DEM = new_DEM.chunk(100)

# Load Seismic Data

In [21]:
array_2pc = np.load('for_DataMesh/for_DataMesh/2perc_disps_sites_c_MDEz_uniform.npy')
array_10pc = np.load('for_DataMesh/for_DataMesh/10perc_disps_sites_c_MDEz_uniform.npy')

In [22]:
df_2pc = pd.DataFrame(array_2pc)
df_2pc.columns = ['ID','Lon','Lat','Uplift','Subsidence','absVLM']
lon_lats = [transformer.transform(x,y) for x,y in zip(df_2pc.Lon,df_2pc.Lat)]
df_2pc['Lon'] = [x[0] for x in lon_lats]
df_2pc['Lat'] = [x[1] for x in lon_lats]

df_10pc = pd.DataFrame(array_10pc)
df_10pc.columns = ['ID','Lon','Lat','Uplift','Subsidence','absVLM']
lon_lats = [transformer.transform(x,y) for x,y in zip(df_10pc.Lon,df_10pc.Lat)]
df_10pc['Lon'] = [x[0] for x in lon_lats]
df_10pc['Lat'] = [x[1] for x in lon_lats]

In [23]:
# Put onto DEM grid
x = new_DEM.x
y = new_DEM.y
X, Y = np.meshgrid(x, y)

array_2pc = griddata((df_2pc['Lon'], df_2pc['Lat']), df_2pc.absVLM, (X, Y), method='linear')
array_10pc = griddata((df_10pc['Lon'], df_10pc['Lat']), df_10pc.absVLM, (X, Y), method='linear')
array_none = array_10pc.copy()
array_none[:,:] = 0

In [24]:
array_2pc = array_2pc.astype(np.float32)
array_2pc = csr_matrix(array_2pc)

array_10pc = array_10pc.astype(np.float32)
array_10pc = csr_matrix(array_10pc)

array_none = array_none.astype(np.float32)
array_none = csr_matrix(array_none)

In [25]:

earthquake_scens = {
    'None':array_none,
    '2pc':array_2pc,
    '10pc':array_10pc
}

# Storm Surge

DataSet: https://data.4tu.nl/articles/_/13392314/1

In [26]:
xr_storm_surge = xr.open_dataset('COAST-RP.nc')
xr_100rp = xr_storm_surge['storm_tide_rp_0100']

In [27]:
well_y = -41.274678
well_x = 174.854143

In [28]:
df_storm_surge = xr_storm_surge.to_dataframe()

In [29]:
df_storm_surge.loc[:,'euclidean'] = (df_storm_surge.station_x_coordinate-well_x)**2+(df_storm_surge.station_y_coordinate-well_y)**2
df_storm_surge = df_storm_surge.sort_values('euclidean').reset_index(drop=True)
storm_tide_rps_dict = df_storm_surge.loc[0,[x for x in df_storm_surge.columns if 'storm_tide_rp' in x]].to_dict()

In [30]:
df_storm_surge

,station_id,storm_tide_rp_0001,storm_tide_rp_0002,storm_tide_rp_0005,storm_tide_rp_0010,storm_tide_rp_0025,storm_tide_rp_0050,storm_tide_rp_0100,storm_tide_rp_0250,storm_tide_rp_0500,storm_tide_rp_1000,station_x_coordinate,station_y_coordinate,euclidean
0,islands_27003,0.536,0.564,0.598,0.620,0.648,0.668,0.687,0.704,0.712,0.722,174.858002,-41.264999,0.000109
1,id_coast_glob_08580,0.534,0.562,0.596,0.619,0.646,0.667,0.686,0.703,0.712,0.720,174.800003,-41.293999,0.003304
2,id_coast_glob_08581,1.069,1.111,1.160,1.192,1.234,1.264,1.294,1.329,1.354,1.378,174.770996,-41.146999,0.023215
3,islands_26834,1.134,1.178,1.228,1.262,1.302,1.333,1.365,1.403,1.431,1.454,174.770996,-41.089001,0.041389
4,id_coast_glob_08579,0.996,1.027,1.064,1.089,1.119,1.141,1.159,1.179,1.193,1.207,175.181000,-41.439999,0.134168
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23221,id_coast_glob_08194,2.208,2.342,2.528,2.668,2.840,2.974,3.110,3.295,3.397,3.465,-179.735992,66.108002,137265.218750
23222,id_coast_glob_08197,0.733,0.794,0.875,0.931,0.997,1.036,1.068,1.109,1.129,1.146,-178.212997,71.528000,137380.843750
23223,id_coast_glob_08198,0.846,0.903,0.978,1.027,1.082,1.123,1.161,1.200,1.217,1.233,-178.856995,70.913002,137697.625000
23224,id_coast_glob_08195,1.096,1.207,1.329,1.400,1.479,1.540,1.618,1.679,1.706,1.724,-179.854004,68.920998,137960.953125


In [31]:
storm_tide_rps_dict = {k.split('_')[-1]:v for k,v in storm_tide_rps_dict.items()}

In [32]:
storm_tide_rps_dict

{'0001': 0.5360000133514404,
 '0002': 0.5640000104904175,
 '0005': 0.5979999899864197,
 '0010': 0.6200000047683716,
 '0025': 0.6480000019073486,
 '0050': 0.6679999828338623,
 '0100': 0.6869999766349792,
 '0250': 0.7039999961853027,
 '0500': 0.7120000123977661,
 '1000': 0.722000002861023}

# Load VLMs

In [33]:
df_vlms = pd.read_csv('Welly_VLM_2018-2023_100m.txt',delimiter='\t')
df_vlms = df_vlms.astype(np.float32)

In [34]:
df_vlms = df_vlms.rename(columns={'  0.000000':'lon','  0.000000.1':'lat'})

In [35]:
df_vlms.set_index(['lat','lon'],inplace=True)
df_vlms.columns = df_vlms.columns.astype(float)
df_vlms = df_vlms.reset_index()

In [36]:
cols = [x for x in df_vlms.columns if x not in ['lon','lat']]

In [37]:
# import dask.array as da
# import numpy as np
# from scipy.interpolate import griddata

# x = new_DEM.x  # Assuming these are your x coordinates
# y = new_DEM.y  # Assuming these are your y coordinates

# X, Y = np.meshgrid(x, y)

# # List to store delayed computations
# delayed_computations = []

# # Function to compute interpolated values for a specific year
# def interpolate_year(year):
#     return griddata((df_vlms['lon'], df_vlms['lat']), df_vlms[year], (X, Y), method='linear').astype(np.float32)

# # Create delayed computations for each year
# for year in cols:
#     delayed_comp = da.from_delayed(delayed(interpolate_year)(year).chunks(200), shape=(len(x), len(y)), dtype=np.float32)
#     delayed_computations.append(delayed_comp)

# # Stack the delayed computations into a Dask array
# xr_vlms = da.stack(delayed_computations, axis=-1)

# # Now you can compute the result
# result = xr_vlms.compute()


In [38]:
# delayed(interpolate_year)(year).chunks(100)

In [39]:
# delayed_computations

In [40]:
# import dask as da

In [41]:
# x = new_DEM.x#np.linspace(df_vlms['lon'].min(), df_vlms['lon'].max(), 100)
# y = new_DEM.y#np.linspace(df_vlms['lat'].min(), df_vlms['lat'].max(), 100)

# X, Y = np.meshgrid(x, y)

# xr_vlms = dask.array.empty((len(x),len(y),len(cols)),chunks=100)

# i = 0
# for year in cols:
#     xr_vlms[:,:,i] = griddata((df_vlms['lon'], df_vlms['lat']), df_vlms[year], (X, Y), method='linear').astype(np.float32)
#     i+=1



In [42]:
# xr_vlms[:,:,i]

In [43]:
# import dask.array as da
# import numpy as np
# from scipy.interpolate import griddata
# from dask import delayed

# x = new_DEM.x  # Assuming these are your x coordinates
# y = new_DEM.y  # Assuming these are your y coordinates

# X, Y = np.meshgrid(x, y)

# # Create an empty Dask array
# xr_vlms = da.empty((len(x), len(y), len(cols)), chunks=(100, 100, 1))

# # Function to compute interpolated values for a specific year and index
# def interpolate_year(year, i):
#     return griddata((df_vlms['lon'], df_vlms['lat']), df_vlms[year], (X, Y), method='linear').astype(np.float32)

# # Delayed computations for each year
# delayed_computations = [delayed(interpolate_year)(year, i) for i, year in enumerate(cols)]

# # Stack the delayed computations outside the loop
# stacked = da.stack(delayed_computations, axis=-1)

# # Assign the stacked result to the Dask array
# xr_vlms[:, :, :] = stacked

# # Now you can compute the result
# result = xr_vlms.compute()


In [44]:
# delayed_comp

In [45]:
# xr_vlms[:,:,i] = griddata((df_vlms['lon'], df_vlms['lat']), df_vlms[year], (X, Y), method='linear').astype(np.float32)

In [46]:
# xr.DataArray(np.empty(len(x),len(y),len(cols))

In [48]:
x = new_DEM.x#np.linspace(df_vlms['lon'].min(), df_vlms['lon'].max(), 100)
y = new_DEM.y#np.linspace(df_vlms['lat'].min(), df_vlms['lat'].max(), 100)

X, Y = np.meshgrid(x, y)

example_year = 2150
Z = griddata((df_vlms['lon'], df_vlms['lat']), df_vlms[2018.0], (X, Y), method='linear')
Z = np.nan_to_num(Z,nan=0)

Z_nax = Z[:,:,np.newaxis]
duplicated_arr = xr.DataArray(np.broadcast_to(Z_nax, (*Z.shape, len(cols))))#.chunk(100)
duplicated_arr = duplicated_arr.astype(np.float32)
duplicated_arr = duplicated_arr.copy()
# new array has dims of lon,lat,time

i = 0
for year in cols:
    duplicated_arr[:,:,i] = griddata((df_vlms['lon'], df_vlms['lat']), df_vlms[year], (X, Y), method='linear').astype(np.float32)
    i+=1

xr_vlms = xr.DataArray(duplicated_arr,coords={"lon":np.array(x),"lat":np.array(y),"years":cols},dims=['lon','lat','years'])

xr_vlm_interp1 = xr_vlms.interpolate_na(dim='lat',method='linear')
xr_vlm_interp2 = xr_vlms.interpolate_na(dim='lon',method='linear')

xr_vlm_interp = (xr_vlm_interp1+xr_vlm_interp2)/2

In [49]:
duplicated_arr = duplicated_arr.rename({'dim_0':'lon','dim_1':'lat','dim_2':'years'})

In [50]:
xr_vlms

<xarray.DataArray (lon: 500, lat: 500, years: 152)>
array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
...
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]], dtype=float32)
Coordinates:
  * lon      (lon) float64 174.7 174.7 174.7 174.7 ... 175.4 175.4 175.4 175.4
  * lat      (lat) float64 -41.52 -41.51 -41.51 -41.51 ... -40.8 -40.8 -40.8
  * years    (years) float64 2.018e+03 2.018e+03 ... 2.023e+03 2.023e+03

In [51]:
del xr_vlms

In [52]:
xr.DataArray(duplicated_arr).chunk(100)

<xarray.DataArray (lon: 500, lat: 500, years: 152)>
dask.array<xarray-<this-array>, shape=(500, 500, 152), dtype=float32, chunksize=(100, 100, 100), chunktype=numpy.ndarray>
Dimensions without coordinates: lon, lat, years

In [54]:
# xr_vlms = xr_vlms.chunk(100)

In [55]:
# import dask.array as da
# from dask import delayed
# from scipy.interpolate import griddata

# # Assuming df_vlms is your DataFrame

# x = new_DEM.x  # Assuming new_DEM is defined somewhere
# y = new_DEM.y

# # Define chunk size (adjust as needed)
# chunk_size = 1  # Start with a small chunk size

# # Create an empty Dask array with float32 data type
# xr_vlms = da.zeros((len(cols), len(y), len(x)), chunks=(chunk_size, len(y), len(x)), dtype=np.float16)

# # Load and process data in chunks
# for i in range(0, len(cols), chunk_size):
#     chunk_cols = cols[i:i+chunk_size]
#     delayed_chunks = []
#     for j, year in enumerate(chunk_cols):
#         delayed_chunk = delayed(griddata)(
#             (df_vlms['lon'], df_vlms['lat']),
#             df_vlms[year],
#             (X, Y),
#             method='linear'
#         )
#         delayed_chunks.append(delayed_chunk)

#     chunk_data = []
#     for delayed_chunk in delayed_chunks:
#         Z = delayed_chunk.compute().astype(np.float16)
#         chunk_data.append(Z)
#         del Z  # Free up memory
        
#     chunk_data = np.stack(chunk_data)
#     xr_vlms[i:i+chunk_size] = chunk_data

# # Compute the final array
# xr_vlms = xr_vlms.compute()

# # Print memory usage
# print("Memory usage: {:.2f} GiB".format(xr_vlms.nbytes / (1024 ** 3)))


In [56]:
# num_elements = 154 * 4000 * 4000
# bytes_per_element = 2  # float32
# total_memory = num_elements * bytes_per_element / (1024 ** 3)  # Convert bytes to GiB
# print("Total memory required: {:.2f} GiB".format(total_memory))


In [57]:
# xr_vlms

In [58]:
# asdf

# Load SLR Data

In [59]:
xr_slr = sp.io.loadmat('../../Ian_relabelled_sites/total_volc_noVLMssp585_medium_confidence_values.mat')

lats = {x:y for x,y in zip(xr_slr['locations'].squeeze(),xr_slr['lat'].squeeze())}
lons = {x:y for x,y in zip(xr_slr['locations'].squeeze(),xr_slr['lon'].squeeze())}

x = new_DEM.x
y = new_DEM.y

years = np.unique(xr_slr['years'])
quantiles = [0.17,0.5,0.83]

X, Y = np.meshgrid(x, y)

example_year = np.min(years)
example_quantile = np.min(quantiles)

In [60]:
file_names = os.listdir("../../Ian_relabelled_sites/")
file_names = [x for x in file_names if ('total' in x)&('medium' in x)&('_noVLM' in x)]

In [61]:
duplicated_array = np.zeros((len(x),len(y),len(years),len(quantiles),len(file_names)))

In [62]:
k = 0
for file in file_names:
    xr_slr = sp.io.loadmat(f'../../Ian_relabelled_sites/{file}')
    
    xr_slr = xr.DataArray(xr_slr['sea_level_change'],
                          coords={'locations':xr_slr['locations'].squeeze(),'years':xr_slr['years'].squeeze(),'quantiles':xr_slr['quantiles'].squeeze()},
                         dims=['locations','years','quantiles'])
    
    df_slr = xr_slr.sel(quantiles=quantiles).to_dataframe('slr')
    
    df_slr['Lat'] = [lats[x] for x in df_slr.reset_index()['locations']]
    df_slr['Lon'] = [lons[x] for x in df_slr.reset_index()['locations']]
    
    # Focus on the wellington region
    df_slr = df_slr[(df_slr.Lon>np.min(new_DEM.x.values))&(df_slr.Lat>np.min(new_DEM.y.values))&(df_slr.Lon<np.max(new_DEM.x.values))&(df_slr.Lat<np.max(new_DEM.y.values))]
    
    df_slr = df_slr.reset_index().drop('locations',axis=1)
    
    i = 0
    for year in years:
        j = 0
        for quantile in quantiles:
            df_slr_inst = df_slr[(df_slr.years==year)&(df_slr.quantiles==quantile)]
            duplicated_array[:,:,i,j,k] = griddata((df_slr_inst['Lon'], df_slr_inst['Lat']), df_slr_inst['slr'], (X, Y), method='linear')
            j+=1
        i+=1
    k+=1

xr_slr = xr.DataArray(duplicated_array,coords={"Lon":np.array(x),"Lat":np.array(y),"years":years,"quantiles":quantiles,"scenarios":file_names},dims=['Lon','Lat','years','quantiles','scenarios'])

filled_x = xr_slr.ffill('Lon').bfill('Lat')
filled_y = xr_slr.ffill('Lat').bfill('Lon')
xr_slr_filled = (filled_x+filled_y)/2
    

# Adding SLR and VLM together Data

In [63]:
# Find a mean of the rates and project forward
xr_vlm_mean = xr_vlm_interp.mean(dim='years',skipna=True)
xr_vlm_mean = xr_vlm_mean.expand_dims({'years':xr_slr.years.values})

year_count = (xr_vlm_mean.years-np.min(xr_vlm_mean.years)) #should really start when the DEM was made for
xr_vlm_forecast = xr_vlm_mean*np.array(year_count)[:,None,None]
xr_vlm_forecast = xr_vlm_forecast.transpose('lon','lat','years')
xr_slr_vlm_adjusted = xr_slr_filled-np.array(xr_vlm_forecast)[:,:,:,None,None]

In [64]:
xr_slr_vlm_adjusted.scenarios

<xarray.DataArray 'scenarios' (scenarios: 5)>
array(['total_volc_noVLMssp119_medium_confidence_values.mat',
       'total_volc_noVLMssp126_medium_confidence_values.mat',
       'total_volc_noVLMssp245_medium_confidence_values.mat',
       'total_volc_noVLMssp370_medium_confidence_values.mat',
       'total_volc_noVLMssp585_medium_confidence_values.mat'], dtype='<U51')
Coordinates:
  * scenarios  (scenarios) <U51 'total_volc_noVLMssp119_medium_confidence_val...

# Flooding the DEM

In [65]:
new_DEM_mask = new_DEM.copy()
new_DEM_mask = new_DEM_mask/new_DEM_mask
new_DEM_mask = new_DEM_mask.fillna(123456789)#where(new_DEM_mask!=float('NaN'),123456789)
new_DEM_mask = new_DEM_mask.where(new_DEM_mask!=1,np.nan)
new_DEM_mask = new_DEM_mask.where(new_DEM_mask!=123456789,1)

In [66]:
(np.empty((len(list(storm_tide_rps_dict.keys())),
         len(list(earthquake_scens.keys())),
         len(years),
         len(file_names),
         len(x),
         len(y))).astype(np.float32)).shape

(10, 3, 14, 5, 500, 500)

In [67]:
masked_xarray = xr.DataArray(np.empty((len(list(storm_tide_rps_dict.keys())),
         len(list(earthquake_scens.keys())),
         len(years),
         len(file_names),
         len(x),
         len(y))).astype(np.float32),dims=['stormsurge','earthquake','years','file','lon','lat'],
                            coords=[list(storm_tide_rps_dict.keys()),list(earthquake_scens.keys()),years,file_names,x,y])

flooded_xarray = xr.DataArray(np.empty((len(list(storm_tide_rps_dict.keys())),
         len(list(earthquake_scens.keys())),
         len(years),
         len(file_names),
         len(x),
         len(y))).astype(np.float32),dims=['stormsurge','earthquake','years','file','lon','lat'],
                            coords=[list(storm_tide_rps_dict.keys()),list(earthquake_scens.keys()),years,file_names,x,y])

In [68]:
masked_array_dict = {}
flooded_dict = {}

for storm_rps,storm_value in storm_tide_rps_dict.items():
    for e_scen,e_array in earthquake_scens.items():
        for year in years:
            for scenario in file_names:
                xr_slr_vlm_adjusted_year = xr_slr_vlm_adjusted.sel(years=year,scenarios=scenario)
                
                new_DEM_flooded_low = new_DEM-np.array(xr_slr_vlm_adjusted_year.sel(quantiles=np.min(quantiles)))/1000-e_array-float(storm_value)
                new_DEM_flooded_low = new_DEM_flooded_low
                flooded_low = new_DEM_flooded_low.where(new_DEM_flooded_low<0,np.nan)*0+1
                
                new_DEM_flooded_mid = new_DEM-np.array(xr_slr_vlm_adjusted_year.sel(quantiles=0.5))/1000-e_array-float(storm_value)
                new_DEM_flooded_mid = new_DEM_flooded_mid
                flooded_mid = new_DEM_flooded_mid.where(new_DEM_flooded_mid<0,np.nan)*0+1
                
                new_DEM_flooded_high = new_DEM-np.array(xr_slr_vlm_adjusted_year.sel(quantiles=np.max(quantiles)))/1000-e_array-float(storm_value)
                new_DEM_flooded_high = new_DEM_flooded_high
                flooded_high = new_DEM_flooded_high.where(new_DEM_flooded_high<0,np.nan)*0+1
                
                flooded = flooded_low.fillna(0)+flooded_mid.fillna(0)+flooded_high.fillna(0)
                flooded = flooded.where(flooded>0,np.nan)
                
                masked_flooded = np.ma.masked_invalid(flooded)
                masked_flooded = (np.max(masked_flooded)-masked_flooded)/(np.max(masked_flooded)-np.min(masked_flooded))
                masked_flooded = masked_flooded[::-1,:]
        
                masked_array_dict.update({
                    (year,scenario,e_scen,storm_rps):masked_flooded
                })
        
                flooded_dict.update({
                    (year,scenario,e_scen,storm_rps):flooded
                })

                masked_xarray.loc[{'stormsurge': storm_rps, 'earthquake': e_scen, 'years': year, 'file': scenario}] = masked_flooded
                flooded_xarray.loc[{'stormsurge': storm_rps, 'earthquake': e_scen, 'years': year, 'file': scenario}] = flooded

     

# Save to the mesh

In [69]:
data_vars = {'mask': masked_xarray, 'data': flooded_xarray}
# Create a Dataset from the dictionary
xr_flooded = xr.Dataset(data_vars)

In [70]:
xr_flooded

<xarray.Dataset>
Dimensions:     (stormsurge: 10, earthquake: 3, years: 14, file: 5, lon: 500,
                 lat: 500)
Coordinates:
  * stormsurge  (stormsurge) <U4 '0001' '0002' '0005' ... '0250' '0500' '1000'
  * earthquake  (earthquake) <U4 'None' '2pc' '10pc'
  * years       (years) int32 2020 2030 2040 2050 2060 ... 2120 2130 2140 2150
  * file        (file) <U51 'total_volc_noVLMssp119_medium_confidence_values....
  * lon         (lon) float64 174.7 174.7 174.7 174.7 ... 175.4 175.4 175.4
  * lat         (lat) float64 -41.52 -41.51 -41.51 -41.51 ... -40.8 -40.8 -40.8
Data variables:
    mask        (stormsurge, earthquake, years, file, lon, lat) float32 nan ....
    data        (stormsurge, earthquake, years, file, lon, lat) float32 nan ....

In [71]:
# datamesh.write_datasource(datasource_id="wellington_dynamic_shoreline_projections_mask",
#                           name="Wellington Casestudy Dynamic Shoreline Projections Mask",
#                           description="Wellington Casestudy Dynamic Shoreline Projections Mask",
#                           data=xr_flooded,
#                           # coordinates={'lon':'longitude','lat':'latitude'},
#                          tags=['demo', 'wellington','lower hutt'],
#                          )


KeyboardInterrupt



# Visualise

In [72]:
cmap = plt.cm.get_cmap('brg')

wellington_coords = [-41.28664, 174.77557]
zoom = 11

def create_map(year,ssp, earthquake,storm_surge):
    token = "pk.eyJ1Ijoic2hhbm5vbi1iZW5ndHNvbiIsImEiOiJja3F1Y2Q0dHEwMzYwMm9wYmtzYzk2bDZuIn0.5jGMyEiJdmXs1HL7x3ThPw" # your mapbox token
    tileurl = 'https://api.mapbox.com/v4/mapbox.satellite/{z}/{x}/{y}@2x.png?access_token=' + str(token)
    m = folium.Map(location=wellington_coords, zoom_start=zoom)
    custom_tile_layer = folium.TileLayer(tiles=tileurl, name='Satellite',attr='Mapbox',overlay=False).add_to(m)

    masked_flooded = masked_array_dict[(year,ssp,earthquake,storm_surge)]
    flooded = flooded_dict[(year,ssp,earthquake,storm_surge)]

    # print(year,ssp)
    
    folium.raster_layers.ImageOverlay(cmap(masked_flooded),
                                      [[flooded.y.values.min(), flooded.x.values.min()],
                                       [flooded.y.values.max(), flooded.x.values.max()]],opacity=.5).add_to(m)

    return(m)
    
# Define dropdown options
dropdown_options1 = ['ssp126','ssp126', 'ssp245','ssp370','ssp585']
dropdown_options2 = years
dropdown_options3 = list(earthquake_scens.keys())
dropdown_options4 = list(storm_tide_rps_dict.keys())

# Define callback function to update map when dropdown value changes
def update_map(year, ssp, earthquake,storm_surge):
    clear_output(wait=True)
    try:
        m = create_map(year, f'total_volc_noVLM{ssp}_medium_confidence_values.mat',earthquake,storm_surge)
        display(m)
    except Exception as e:
        print(f"Error occurred: {e}")

# Create interactive dropdowns
interact(update_map, year=dropdown_options2, ssp=dropdown_options1,earthquake=dropdown_options3,storm_surge=dropdown_options4)


C:\Users\shannonb\AppData\Local\Temp\ipykernel_18388\3718186746.py:1: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = plt.cm.get_cmap('brg')


interactive(children=(Dropdown(description='year', options=(2020, 2030, 2040, 2050, 2060, 2070, 2080, 2090, 21…

<function __main__.update_map(year, ssp, earthquake, storm_surge)>

In [ ]:
masked_flooded.shape

In [ ]:
array_10pc.shape